# Daily Solar Curve Analysis

This notebook analyzes daily solar generation curves from CAISO data, comparing summer vs winter patterns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gridstatus
from datetime import datetime
import os

# Setup
DATA_DIR = '../data/raw'
os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
# Load CAISO data (cached from previous analysis)
start_date = '2024-01-01'
end_date = '2025-01-01'
cache_file = f'{DATA_DIR}/solar_fuel_mix_{start_date}_{end_date}.csv'

if os.path.exists(cache_file):
    fuel_mix = pd.read_csv(cache_file, parse_dates=['Time'])
    fuel_mix['Time'] = pd.to_datetime(fuel_mix['Time'], utc=True)
    fuel_mix = fuel_mix[fuel_mix['Time'].dt.year == 2024]
    print(f"✓ Loaded cached data: {len(fuel_mix)} rows")
else:
    print("❌ Cache file not found. Please run the data fetching notebook first.")

In [ ]:
# Pick summer and winter days
summer_day = '2024-06-29'
winter_day = '2024-12-30'

# Filter CAISO data for those days
summer_caiso = fuel_mix[fuel_mix['Time'].dt.date == pd.to_datetime(summer_day).date()].copy()
winter_caiso = fuel_mix[fuel_mix['Time'].dt.date == pd.to_datetime(winter_day).date()].copy()

# Convert to Pacific Time
summer_caiso['Time_Local'] = summer_caiso['Time'].dt.tz_convert('America/Los_Angeles')
winter_caiso['Time_Local'] = winter_caiso['Time'].dt.tz_convert('America/Los_Angeles')

# Create hour of day for easier comparison
summer_caiso['Hour'] = summer_caiso['Time_Local'].dt.hour + summer_caiso['Time_Local'].dt.minute / 60
winter_caiso['Hour'] = winter_caiso['Time_Local'].dt.hour + winter_caiso['Time_Local'].dt.minute / 60

In [ ]:
# Plot summer vs winter daily curves
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(summer_caiso['Hour'], summer_caiso['Solar'], 
        label=f'Summer ({summer_day})', linewidth=2, color='orange', marker='o', markersize=3)
ax.plot(winter_caiso['Hour'], winter_caiso['Solar'], 
        label=f'Winter ({winter_day})', linewidth=2, color='blue', marker='o', markersize=3)

ax.set_xlabel('Hour of Day (Pacific Time)')
ax.set_ylabel('Solar Generation (MW)')
ax.set_title('CAISO Solar Generation: Summer vs Winter Day')
ax.legend()
ax.set_ylim(bottom=0)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 24)

plt.tight_layout()
plt.show()

In [ ]:
# Print summary stats
print(f"\n📊 Summary for {summer_day}:")
print(f"Peak: {summer_caiso['Solar'].max():.0f} MW at {summer_caiso.loc[summer_caiso['Solar'].idxmax(), 'Time_Local'].strftime('%I:%M %p %Z')}")
print(f"Total: {(summer_caiso['Solar'] * (5/60)).sum():.0f} MWh")

print(f"\n📊 Summary for {winter_day}:")
print(f"Peak: {winter_caiso['Solar'].max():.0f} MW at {winter_caiso.loc[winter_caiso['Solar'].idxmax(), 'Time_Local'].strftime('%I:%M %p %Z')}")
print(f"Total: {(winter_caiso['Solar'] * (5/60)).sum():.0f} MWh")

print(f"\n📊 Summer vs Winter:")
print(f"Peak ratio (summer/winter): {summer_caiso['Solar'].max() / winter_caiso['Solar'].max():.2f}x")
print(f"Total ratio (summer/winter): {(summer_caiso['Solar'] * (5/60)).sum() / (winter_caiso['Solar'] * (5/60)).sum():.2f}x")